In [12]:
import numpy as np
from pathlib import Path
import sys
from PIL import Image
import os

import torchvision.transforms as transforms

In [13]:
# Constructs a path to a directory that contains dataloader.py and plotCreator.py
module_path = str(Path.cwd().parents[0] / "methods")

# Checks to see if the directory is already in sys.path to avoid adding it multiple times.
if module_path not in sys.path:
    sys.path.append(module_path)

# Imports all the functions defined in the dataloader.py
from dataloader import *

# Creates two file paths pointing to two HDF5 files
data_path0 = str(Path.cwd().parents[0] / "data" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0] / "data" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

# Reads the two HDF5 data files and creates two NumPy arrays
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

In [14]:
def save_images(bh_images, sph_images):
    bh_folder = "black_holes"
    sph_folder = "sphalerons"

    if not os.path.exists(bh_folder):
        os.makedirs(bh_folder)

    if not os.path.exists(sph_folder):
        os.makedirs(sph_folder)

    for idx, img in enumerate(bh_images):
        image = Image.fromarray(np.uint8(np.moveaxis(img, 0, -1)))  # Convert (C, H, W) to (H, W, C)
        image.save(f"{bh_folder}/black_hole_{idx + 1}.png")
        
    for idx, img in enumerate(sph_images):
        image = Image.fromarray(np.uint8(np.moveaxis(img, 0, -1)))  # Convert (C, H, W) to (H, W, C)
        image.save(f"{sph_folder}/sphaleron_{idx + 1}.png")


In [19]:
def preprocess_images(images):
    processed_images = []
    for img in images:
        reshaped_image = img.reshape(3, 50, 50)
        image = Image.fromarray(np.uint8(reshaped_image.transpose(1, 2, 0) * 255))
        image = np.array(image.resize((50, 50)))
        image = np.moveaxis(image, -1, 0)
        processed_images.append(image)
    return processed_images

# Define the same transformations as used in training
transform = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def load_and_preprocess_images(bhArray, sphArray):
    bh_images = []
    sph_images = []
    for img in bhArray:
        image = Image.fromarray(np.uint8(np.moveaxis(img, 0, -1)))  # Convert (C, H, W) to (H, W, C)
        image = transform(image)
        bh_images.append(image)
    for img in sphArray:
        image = Image.fromarray(np.uint8(np.moveaxis(img, 0, -1)))  # Convert (C, H, W) to (H, W, C)
        image = transform(image)
        sph_images.append(image)
    return bh_images, sph_images

def get_sample_images(bhArray, sphArray, num_images):
    np.random.seed(42)
    bh_indices = np.random.choice(len(bhArray), num_images, replace=False)
    sph_indices = np.random.choice(len(sphArray), num_images, replace=False)

    bh_sample = bhArray[bh_indices]
    sph_sample = sphArray[sph_indices]

    bh_images, sph_images = load_and_preprocess_images(bh_sample, sph_sample)

    return bh_images, sph_images


num_images = 10
bh_sample_images, sph_sample_images = get_sample_images(bhArray, sphArray, num_images)

sample_image = bh_sample_images[0]  # Change the index to display other images
print(sample_image.shape)
Image.fromarray(np.uint8(sample_image).transpose(1, 2, 0))

# Call the save_images function to save the images
save_images(bh_sample_images, sph_sample_images)


TypeError: Cannot handle this data type: (1, 1, 50), |u1